In [32]:
import customtkinter as ctk
import tkinter.messagebox as tkmb
import nltk
from nltk.corpus import wordnet
import random
import string
import pyttsx3
from collections import Counter

# Download required NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize Text-to-Speech engine
engine = pyttsx3.init()

# List of data science-related words for guessing game
data_science_words = [
    "machine learning", "data analysis", "algorithm", "data cleaning", "normalization", 
    "neural network", "regression", "classification", "dataset", "null value", "standardization",
    "feature engineering", "big data", "deep learning", "artificial intelligence",
    "supervised learning", "feature", "reinforcement learning", "data visualization",
    "clustering", "decision tree", "data mining", "overfitting", "dataframe", 
    "training data", "test data", "overfitting", "accuracy", "precision",
    "confusion matrix", "logistic regression", "outlier"
]

# -------------------- SPLASH SCREEN --------------------
def show_splash_screen(app):
    splash_frame = ctk.CTkFrame(app, width=850, height=700, fg_color="lightblue")
    splash_frame.pack_propagate(False)  # Prevent the frame from resizing
    
    splash_label = ctk.CTkLabel(splash_frame, text="Welcome to the Dictionary & Text Summarizer App!", font=("Arial", 24, "bold"), text_color="black")
    splash_label.pack(pady=250)  # Adjust the padding to center the label vertically
    
    splash_frame.pack()

    # Display splash screen for 3 seconds
    app.update()
    time.sleep(3)
    
    splash_frame.destroy()  # Remove the splash screen after 3 seconds

# -------------------- DICTIONARY FUNCTIONS --------------------
def get_meanings(word):
    synsets = wordnet.synsets(word)
    meanings = list({syn.definition() for syn in synsets})
    return meanings or ["No meanings found."]

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return list(synonyms) or ["No synonyms found."]

def get_antonyms(word):
    antonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            for ant in lemma.antonyms():
                antonyms.add(ant.name().replace('_', ' '))
    return list(antonyms) or ["No antonyms found."]

def lookup_word():
    word = entry_word.get().strip()
    if not word:
        tkmb.showwarning("Input Error", "Please enter a word.")
        return

    dict_output.delete(1.0, ctk.END)

    meanings = get_meanings(word)
    synonyms = get_synonyms(word)
    antonyms = get_antonyms(word)

    output = "📘 MEANINGS\n===========\n"
    output += "\n".join(f"{i+1}. {m}" for i, m in enumerate(meanings))
    output += "\n\n🟦 SYNONYMS\n============\n"
    output += ", ".join(synonyms)
    output += "\n\n🟥 ANTONYMS\n============\n"
    output += ", ".join(antonyms)

    dict_output.insert(ctk.END, output)

def pronounce_word():
    word = entry_word.get().strip()
    if word:
        engine.say(word)
        engine.runAndWait()
    else:
        tkmb.showwarning("Input Error", "Please enter a word to pronounce.")

def clear_dict_text():
    entry_word.delete(0, ctk.END)
    dict_output.delete(1.0, ctk.END)

def word_of_the_day():
    words = ["serendipity", "ephemeral", "labyrinth", "eloquence", "serenity"]
    return random.choice(words)

# -------------------- WORD GUESSING GAME --------------------
import os

class WordGuessingGame:
    def __init__(self, frame):
        self.frame = frame
        self.score = 0
        self.high_score = self.load_high_score()
        self.setup_ui()
        self.new_round()

    def load_high_score(self):
        if os.path.exists("high_score.txt"):
            with open("high_score.txt", "r") as f:
                try:
                    return int(f.read().strip())
                except ValueError:
                    return 0
        return 0

    def save_high_score(self):
        with open("high_score.txt", "w") as f:
            f.write(str(self.high_score))

    def setup_ui(self):
        self.label = ctk.CTkLabel(self.frame, text="", font=("Arial", 18, "bold"), text_color="blue")
        self.label.pack(pady=20)

        self.guess_entry = ctk.CTkEntry(self.frame, placeholder_text="Enter your guess", font=("Arial", 14))
        self.guess_entry.pack(pady=10)

        btn_frame = ctk.CTkFrame(self.frame, fg_color="transparent")
        btn_frame.pack(pady=5)

        self.submit_button = ctk.CTkButton(btn_frame, text="Submit Guess", command=self.check_guess,
                                           font=("Arial", 14), fg_color="#27ae60")
        self.submit_button.grid(row=0, column=0, padx=5)

        self.pass_button = ctk.CTkButton(btn_frame, text="Pass", command=self.pass_word, font=("Arial", 14), fg_color="#7f8c8d")
        self.pass_button.grid(row=0, column=1, padx=5)

        self.next_button = ctk.CTkButton(btn_frame, text="Next Word", command=self.new_round, font=("Arial", 14), fg_color="#f39c12")
        self.next_button.grid(row=0, column=2, padx=5)
        self.next_button.configure(state="disabled")

        self.attempts_label = ctk.CTkLabel(self.frame, text="", font=("Arial", 14), text_color="red")
        self.attempts_label.pack(pady=5)

        self.score_label = ctk.CTkLabel(self.frame, text="Score: 0", font=("Arial", 14), text_color="purple")
        self.score_label.pack()

        self.high_score_label = ctk.CTkLabel(self.frame, text=f"High Score: {self.high_score}", font=("Arial", 14), text_color="orange")
        self.high_score_label.pack()

        self.feedback_label = ctk.CTkLabel(self.frame, text="", font=("Arial", 14), text_color="blue")
        self.feedback_label.pack(pady=10)
        
        self.end_game_button = ctk.CTkButton(self.frame, text="End Game", command=self.end_game, font=("Arial", 14), fg_color="#c0392b")
        self.end_game_button.pack(pady=10)

    def reveal_letters(self, word):
        revealed = set()
        clean_word = word.replace(" ", "")
        word_len = len(clean_word)

        if word_len > 0:
            revealed.add(clean_word[0])
        if word_len > 1:
            revealed.add(clean_word[1])
        if word_len % 2 == 1:
            revealed.add(clean_word[word_len // 2])
        else:
            revealed.add(clean_word[word_len // 2 - 1])
            revealed.add(clean_word[word_len // 2])
        if word_len > 2:
            revealed.add(clean_word[-1])
        if word_len > 3:
            revealed.add(clean_word[-2])

        result = ""
        i = 0
        for char in word:
            if char == " ":
                result += " "
            elif clean_word[i] in revealed:
                result += clean_word[i]
                i += 1
            else:
                result += "_"
                i += 1
        return result

    def check_guess(self):
        guess = self.guess_entry.get().lower().strip()
        if not guess:
            tkmb.showwarning("Input Error", "Please enter a guess.")
            return

        if guess.replace(" ", "") == self.word.replace(" ", ""):
            self.score += 1
            self.feedback_label.configure(text=f"🎉 Correct! The word was: {self.word}", text_color="green")
            self.submit_button.configure(state="disabled")
            self.pass_button.configure(state="disabled")
            self.next_button.configure(state="normal")
            self.update_score()
        else:
            self.attempts -= 1
            self.attempts_label.configure(text=f"Attempts left: {self.attempts}")
            if self.attempts <= 0:
                self.feedback_label.configure(text=f"❌ Out of attempts. The word was: {self.word}", text_color="red")
                self.submit_button.configure(state="disabled")
                self.pass_button.configure(state="disabled")
                self.next_button.configure(state="normal")
            else:
                self.feedback_label.configure(text="Incorrect guess. Try again!", text_color="orange")

    def pass_word(self):
        self.feedback_label.configure(text=f"⏭️ You passed. The word was: {self.word}", text_color="gray")
        self.submit_button.configure(state="disabled")
        self.pass_button.configure(state="disabled")
        self.next_button.configure(state="normal")

    def new_round(self):
        self.word = random.choice(data_science_words).lower()
        self.revealed_word = self.reveal_letters(self.word)
        self.attempts = 3

        self.label.configure(text=f"Guess the word: {self.revealed_word}")
        self.attempts_label.configure(text=f"Attempts left: {self.attempts}")
        self.feedback_label.configure(text="")
        self.guess_entry.delete(0, ctk.END)
        self.submit_button.configure(state="normal")
        self.pass_button.configure(state="normal")
        self.next_button.configure(state="disabled")

    def update_score(self):
        self.score_label.configure(text=f"Score: {self.score}")
        if self.score > self.high_score:
            self.high_score = self.score
            self.high_score_label.configure(text=f"High Score: {self.high_score}")
            self.save_high_score()

    
    def end_game(self):
        self.feedback_label.configure(text=f"Game Over! Your final score is: {self.score}", text_color="purple")
        self.submit_button.configure(state="disabled")
        self.next_button.configure(state="disabled")
        self.pass_button.configure(state="disabled")
        self.end_game_button.configure(state="disabled")
        self.guess_entry.configure(state="disabled")

# -------------------- TEXT SUMMARIZER --------------------


def summarize_text(text, max_words=50):
    translator = str.maketrans('', '', string.punctuation)
    cleaned_text = text.translate(translator).lower()
    words = cleaned_text.split()
    sentences = text.split('.')

    word_freq = Counter(words)
    sentence_scores = {}
    for sentence in sentences:
        sentence_words = sentence.split()
        score = sum(word_freq.get(word.lower(), 0) for word in sentence_words)
        sentence_scores[sentence] = score

    sorted_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    summary = ''
    total_words = 0

    for sent in sorted_sentences:
        word_count = len(sent.split())
        if total_words + word_count <= max_words:
            summary += sent.strip() + ". "
            total_words += word_count
        if total_words >= max_words:
            break

    return summary.strip() or "Summary could not be generated."

def summarize_and_display():
    input_text = summarizer_input.get("1.0", ctk.END).strip()
    if not input_text or len(input_text.split()) < 10:
        tkmb.showwarning("Input Error", "Please enter a longer text (at least 10 words).")
        return

    summary = summarize_text(input_text, max_words=50)
    summarizer_output.delete("1.0", ctk.END)
    summarizer_output.insert(ctk.END, summary)


# -------------------- MODE SWITCH --------------------
def switch_mode(mode):
    if mode == "Dictionary":
        dict_frame.pack(fill=ctk.BOTH, expand=True, padx=20, pady=10)
        summarizer_frame.pack_forget()
        guessing_frame.pack_forget()
    elif mode == "Summarizer":
        dict_frame.pack_forget()
        summarizer_frame.pack(fill=ctk.BOTH, expand=True, padx=20, pady=10)
        guessing_frame.pack_forget()
    else:
        dict_frame.pack_forget()
        summarizer_frame.pack_forget()
        guessing_frame.pack(fill=ctk.BOTH, expand=True, padx=20, pady=10)

# -------------------- MAIN UI --------------------
def launch_main_app():
    global entry_word, dict_output, summarizer_input, summarizer_output
    global dict_frame, summarizer_frame, guessing_frame

    app = ctk.CTk()
    app.title("Dictionary & Summarizer App")
    app.geometry("850x700")
    ctk.set_appearance_mode("light")
    ctk.set_default_color_theme("green")

    # Show the splash screen before loading the main app
    show_splash_screen(app)

    title = ctk.CTkLabel(app, text="📚 Dictionary & Text Summarizer", font=("Arial", 26, "bold"), text_color="blue")
    title.pack(pady=15)

    switch_frame = ctk.CTkFrame(app, fg_color="transparent")
    switch_frame.pack(pady=5)

    btn_dict = ctk.CTkButton(switch_frame, text="Dictionary", command=lambda: switch_mode("Dictionary"),
                             fg_color="#1abc9c", font=("Arial", 14))
    btn_dict.pack(side=ctk.LEFT, padx=10)

    btn_summarizer = ctk.CTkButton(switch_frame, text="Summarizer", command=lambda: switch_mode("Summarizer"),
                                   fg_color="#2980b9", font=("Arial", 14))
    btn_summarizer.pack(side=ctk.LEFT, padx=10)

    btn_guess = ctk.CTkButton(switch_frame, text="Guessing Game", command=lambda: switch_mode("Guessing"),
                              fg_color="#e74c3c", font=("Arial", 14))
    btn_guess.pack(side=ctk.LEFT, padx=10)

    # Dictionary Frame
    dict_frame = ctk.CTkFrame(app, fg_color="#e8f4fc")

    word_label = ctk.CTkLabel(dict_frame, text="Word of the Day: " + word_of_the_day(),
                              font=("Arial", 18, "bold"))
    word_label.pack(pady=10)

    input_row = ctk.CTkFrame(dict_frame, fg_color="#e8f4fc")
    input_row.pack(pady=10)

    entry_word = ctk.CTkEntry(input_row, placeholder_text="Enter a word", font=("Arial", 14), width=300)
    entry_word.grid(row=0, column=0, padx=10)

    btn_search = ctk.CTkButton(input_row, text="🔍 Search", command=lookup_word, font=("Arial", 14))
    btn_search.grid(row=0, column=1, padx=5)

    btn_clear = ctk.CTkButton(input_row, text="❌ Clear", command=clear_dict_text, fg_color="#e74c3c", font=("Arial", 14))
    btn_clear.grid(row=0, column=2, padx=5)

    btn_pronounce = ctk.CTkButton(input_row, text="🔊 Pronounce", command=pronounce_word, fg_color="#8e44ad", font=("Arial", 14))
    btn_pronounce.grid(row=0, column=3, padx=5)

    dict_output = ctk.CTkTextbox(dict_frame, wrap=ctk.WORD, font=("Arial", 14), height=400)
    dict_output.pack(padx=20, pady=10, fill=ctk.BOTH, expand=True)

    # Guessing Game Frame
    guessing_frame = ctk.CTkFrame(app, fg_color="#fff7e6")
    start_game_button = ctk.CTkButton(guessing_frame, text="Start Guessing Game",
                                      command=lambda: WordGuessingGame(guessing_frame),
                                      font=("Arial", 16, "bold"))
    start_game_button.pack(pady=20)

    # Text Summarizer Frame
    summarizer_frame = ctk.CTkFrame(app, fg_color="#f0f3f4")
    summarizer_label = ctk.CTkLabel(summarizer_frame, text="Enter Text to Summarize", font=("Arial", 18, "bold"))
    summarizer_label.pack(pady=10)

    summarizer_input = ctk.CTkTextbox(summarizer_frame, wrap=ctk.WORD, font=("Arial", 14), height=150)
    summarizer_input.pack(padx=20, pady=10, fill=ctk.BOTH, expand=True)

    summarize_button = ctk.CTkButton(summarizer_frame, text="Summarize", command=summarize_and_display,
                                     font=("Arial", 14), fg_color="#2980b9")
    summarize_button.pack(pady=10)

    summarizer_output = ctk.CTkTextbox(summarizer_frame, wrap=ctk.WORD, font=("Arial", 14), height=200)
    summarizer_output.pack(padx=20, pady=10, fill=ctk.BOTH, expand=True)

    dict_frame.pack(fill=ctk.BOTH, expand=True, padx=20, pady=10)
    app.mainloop()

# -------------------- RUN --------------------
def main():
    launch_main_app()

if __name__ == "__main__":
    main()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shravani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Shravani\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


NameError: name 'time' is not defined